In [ ]:
import random

import notears.utils
from bnlearn import bnlearn
from notears.linear import notears_linear
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from baseDS import Graph, Generic
from notears import utils
import pandas as pd
import numpy as np
import bnlearn as bn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

LR = LogisticRegression(random_state=0)

# Ground truth definition using a logit (sigmoid) function
def log_transformation(params0, params1, params2, params3):
    sum = params0 * 2 + params1 - params2 + params3 + random.randint(0,1)
    y = 1 / (1 + np.exp(-sum))
    #1 model with 4 x binary, with binary y with logistic regression
    #rank random forest, linear regression
    #Hot code categorical examples
    #1 - 0-10
    #2 - 10-25
    #instead of bionomal arugment would need categorical argument returning one of either of the values
    y = 1 if y > 0.5 else 0
    return y

# DAG setup - no interaction
def setup_dag():
    Prior1 = Generic(name="A", function=np.random.binomial, arguments={"n":1, "p":0.5})
    Prior2 = Generic(name="B", function=np.random.binomial, arguments={"n":1, "p":0.5})
    Prior3 = Generic(name="C", function=np.random.binomial, arguments={"n":1, "p":0.5})
    Prior4 = Generic(name="D", function=np.random.binomial, arguments={"n":1, "p":0.5})
    Node1 = Generic(name="E", function=log_transformation, arguments={"params0": Prior1, "params1": Prior2, "params2": Prior3, "params3": Prior4})

    listNodes = [Prior1, Prior2, Prior3, Prior4, Node1]
    my_graph = Graph("Logistic Regression - Real-world", listNodes)

    # simulate data for training and testing, with study-specific sample sizes
    train = my_graph.simulate(num_samples=10000, csv_name="train")
    test = my_graph.simulate(num_samples=5000, csv_name="test")

setup_dag()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# import the saved training data
train_data = pd.read_csv("train.csv")
train_data_numpy = train_data.to_numpy()
x_train = train_data.iloc[:, 0:4].to_numpy().reshape([-1, 4]) # num predictors
print("x_train", x_train.shape)
y_train = train_data.iloc[:, 4].to_numpy().reshape([-1]).ravel() # outcome
print("y_train", y_train.shape)

# Sklearn - Construct logistic regression model and fit using simulated training data
def model_training(x_train, y_train):
    LR.fit(x_train, y_train)
    print("Coefficient: ", LR.coef_)
    print("Intercept: ", LR.intercept_)

model_training(x_train, y_train)

# import the saved testing data
test_data = pd.read_csv("test.csv")
x_test = test_data.iloc[:, 0:4].to_numpy().reshape([-1, 4])
print("x_test", x_test.shape)
y_test = test_data.iloc[:, 4].to_numpy().reshape([-1])
print("y_test", y_test.shape)

def model_evaluate(LR):
    y_pred = LR.predict(x_test)
    y_pred_proba = LR.predict_proba(x_test)[:,1]
    LR.score(x_test, y_pred)
    print("Report: Output evaluation measures")

    # Return the evaluation score (R^2) of the model on the testing data
    print("Accuracy of 100: ", accuracy_score(y_test, y_pred)*100)
    score = LR.score(x_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred_proba)*100
    print("AUC ROC curve: ", auc_roc)
    mae = mean_absolute_error(y_test, y_pred)
    print("Mean Absolute Error: ", mae)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error: ", mse)

    print("Confusion Matrix:")
    cm = metrics.confusion_matrix(y_test, y_pred)
    print(cm)

    print(classification_report(y_test, y_pred))

    plt.figure(figsize=(9, 9))
    sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square=True, cmap='Greens_r');
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');
    all_sample_title = 'Accuracy Score: {0}'.format(score)
    plt.title(all_sample_title, size=15);
    plt.show()

model_evaluate(LR)

def bnlearn_sampling():
    edges = [('A', 'E'),
             ('B', 'E'),
             ('C', 'E'),
             ('D', 'E')]
    DAG = bnlearn.make_DAG(edges)
    bn.plot(DAG)
    print(DAG['adjmat'])
    print(type(train_data))
    DAG = bn.parameter_learning.fit(DAG, train_data, methodtype='maximumlikelihood')
    bn_sample = bn.sampling(DAG, n=100)
    np.savetxt('Z_est.csv', bn_sample, delimiter=',')

bnlearn_sampling()

def notears_sampling():
    nt_sampling = notears_linear(train_data_numpy[0:100], lambda1=0.01, loss_type='l2')
    assert utils.is_dag(nt_sampling)
    np.savetxt('W_est.csv', nt_sampling, delimiter=',')

notears_sampling()

no_tears_sample = pd.read_csv('W_est.csv')
print("no tears samples")
print(no_tears_sample)
bn_learn_sample = pd.read_csv('Z_est.csv')
print("bn learn samples")
print(bn_learn_sample)


def decision_tree():
    clf = DecisionTreeClassifier(random_state=0)
    clf = clf.fit(bn_learn_sample.iloc[:,0:4], bn_learn_sample.iloc[:,4])
    y_pred = clf.predict(x_test)

    print("Accuracy of bn_learn (DecisionTree)", metrics.accuracy_score(y_test,y_pred))

    clf = clf.fit(no_tears_sample.iloc[:,0:4], no_tears_sample.iloc[:,4])
    y_pred = clf.predict(x_test)

    print("Accuracy of no_tears (DecisionTree)", metrics.accuracy_score(y_test,y_pred))

decision_tree()

def random_forest():
    rf = RandomForestClassifier(random_state=0)
    rf = rf.fit(bn_learn_sample.iloc[:,0:4], bn_learn_sample.iloc[:,4])
    n_scores = cross_val_score(rf, bn_learn_sample.iloc[:,0:4], bn_learn_sample.iloc[:,4], scoring='accuracy')
    print('Accuracy of bn_learn (RandomForestClassifier):', np.average(n_scores), '%.')

    rf = rf.fit(no_tears_sample.iloc[:, 0:4], no_tears_sample.iloc[:, 4])
    n_scores = cross_val_score(rf, x_test, y_test, scoring='accuracy')
    print('Accuracy of no_tears (RandomForestClassifier):', np.average(n_scores), '%.')

random_forest()